In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

import json

import pandas as pd

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv('datasets/staging_test_set.csv')

In [ ]:
with open('templates/prompt_template.txt', 'r') as template_file:
    template = template_file.read()

In [ ]:
with open('EDAM/edam_topics.txt', 'r') as edam_file:
    full_edam_topics = edam_file.readlines()

full_edam_topics = [topic.strip() for topic in full_edam_topics]

In [ ]:
# Add EDAM topics to prompt template

formatted_topics = "\n".join(full_edam_topics)
template = template.replace("<topics>", formatted_topics)


## Testing Meditron 7b 

Note: You need a lot of RAM for this

In [ ]:
with open("config.json", "r") as config_file:
    config = json.load(config_file)

In [ ]:
hf_token = config['api_keys']['huggingface']

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("epfl-llm/meditron-7b", token=hf_token)

tokenizer = AutoTokenizer.from_pretrained("./meditron-7b-tokenizer")

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("epfl-llm/meditron-70b", token=hf_token)
# model = AutoModelForCausalLM.from_pretrained("epfl-llm/meditron-7b", token=hf_token)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained("./meditron-7b-model")
model

In [ ]:
model = model.to_bettertransformer()

In [ ]:
# model.save_pretrained("meditron-7b-model", from_pt=True)
# tokenizer.save_pretrained("meditron-7b-tokenizer", from_pt=True)

In [ ]:
random_sample = dataset.sample(n=1)
random_sample

In [ ]:
prompt = template.replace('<abstract>', random_sample['Abstract'].values[0])
prompt


In [ ]:
model_inputs = tokenizer(template, padding=True, return_tensors="pt")

In [ ]:
model_inputs

In [ ]:
outputs = model(**model_inputs)

In [ ]:
predicted_label_classes = outputs.logits.argmax(-1)
predicted_label_classes

In [ ]:
tokenizer.decode(outputs)